In [1]:
!pip install konlpy

  Using cached konlpy-0.6.0-py2.py3-none-any.whl (19.4 MB)
     |████████████████████████████████| 381 kB 11.0 MB/s eta 0:00:01


In [1]:
import pandas as pd
import re
from konlpy.tag import Okt,Mecab
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score,f1_score
from lightgbm import LGBMClassifier

In [2]:
# 데이터 불러오기
train = pd.read_csv('train_data.csv')

In [3]:
# 형태소 분석기(Okt) 불러오기 
okt=Okt()

In [4]:
# 조사, 어미, 구두점 제거
def func(text):
    clean = []
    for word in okt.pos(text, stem=True): #어간 추출
        if word[1] not in ['Josa', 'Eomi', 'Punctuation']: #조사, 어미, 구두점 제외 
            clean.append(word[0])
    
    
    return " ".join(clean) 

train['title'] = train['title'].apply(lambda x : func(x))

In [24]:
train2 = pd.read_csv('train_data.csv')

In [23]:
train

,index,title,topic_idx
0,0,인천 → 핀란드 항공기 결항 휴가 철 여행객 분통,4
1,1,실리콘밸리 넘어서다 구글 15조원 들이다 美 전역 거점 화,4
2,2,이란 외무 긴장 완화 해결 책 미국 경제 전쟁 멈추다 것,4
3,3,NYT 클린턴 측근 韓 기업 특수 관계 조명 공과 사 맞다 물리다 종합,4
4,4,시진핑 트럼프 중미 무역 협상 조속 타결 희망,4
...,...,...,...
45649,45649,KB 금융 미국 IB 스티펠 제휴 선진국 시장 공략,1
45650,45650,1 보 서울시 교육청 신종 코로나 확산 개학 연기 휴업 검토,2
45651,45651,게시판 키우다 증권 2020 키우다 영웅 전 실전 투자 대회,1
45652,45652,답변 하다 배 기동 국립 중앙 박물관 장,2


In [5]:
# tf-idf를 이용한 벡터화
def split(text):
    tokens_ko = text.split()
    return tokens_ko

tfidf_vect = TfidfVectorizer(tokenizer=split)
tfidf_vect.fit(train['title'])
tfidf_matrix_train = tfidf_vect.transform(train['title'])

/Users/bangsumin/opt/anaconda3/lib/python3.8/site-packages/sklearn/feature_extraction/text.py:489: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn("The parameter 'token_pattern' will not be used"


In [6]:
# train/valid 데이터 셋 나누기.
def split_dataset(tfidf,df):
    X_data = tfidf
    y_data = df['topic_idx']

    # stratify=y_data Stratified 기반 분할, train 데이터의 30%를 평가 데이터 셋으로 사용. (70% 데이터 학습에 사용)
    X_train, X_test, y_train, y_test = \
    train_test_split(X_data, y_data, test_size=0.3, random_state=42, stratify=y_data)

    
    return (X_train, X_test, y_train, y_test)

X_train, X_test, y_train, y_test = split_dataset(tfidf_matrix_train,train)

In [7]:
lgbm = LGBMClassifier(random_state = 42)
lgbm.fit(X_train,y_train)

LGBMClassifier(random_state=42)

In [8]:
pred = lgbm.predict(X_test)
accuracy = accuracy_score(y_test,pred)

print('정확도', accuracy)

정확도 0.810980506680295


In [9]:
test = pd.read_csv('test_data.csv')

In [10]:
test['title'] = test['title'].apply(lambda x : func(x)) 

In [11]:
tfidf_matrix_test = tfidf_vect.transform(test['title'])

In [12]:
pred = lgbm.predict(tfidf_matrix_test)

In [13]:
submission = pd.read_csv('sample_submission.csv')

In [14]:
submission['topic_idx'] = pred
submission.head()

,index,topic_idx
0,45654,2
1,45655,3
2,45656,2
3,45657,2
4,45658,3


In [15]:
submission.to_csv('submission2.csv',index=False)